In [1]:
# STOCK PRICE 1: Get PSE stock price using requests library

import requests
import pandas as pd
import dateutil.parser

directory = r'C:\Users\ojell\Desktop\Oj\_Thesis\Data\stock_price'

base_url = r'https://pselookup.vrymel.com/api'
sym = 'AC'
per = ['2021-01-04', '2021-01-05']

response = requests.get(base_url + f'/stocks/{sym}/history/{per[0]}/{per[1]}')
data_json = response.json()
df = pd.DataFrame(data_json['history'])
df['timestamp'] = df['timestamp'].apply(lambda x: dateutil.parser.isoparse(x).strftime('%Y-%m-%d %H:%M:%S'))
#df.to_csv(directory + r'\ayala_test.csv', index=False)   # Comment out to check

In [ ]:
### from bs4 import BeautifulSoup
import requests
import pandas as pd


url = r'https://www.bworldonline.com/category/top-stories/'

headers = headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
cont = requests.get(url, headers).content

soup = BeautifulSoup(cont, "html.parser")

test = soup.find("ul", {"id": "menu-header-menu-1"}).find_all("a")

category = ["Top Stories"]

ctgry_url = []
for item in range(0, len(test)):
    for i in range(0, len(category)):
        if test[item].text.lower() == category[i].lower():
            ctgry_url.append(test[item].get("href"))
            
#print(ctgry_url)

In [2]:
# STOCK PRICE 2: Get PSE stock price using requests library from API
#                https://pselookup.vrymel.com/api
import requests
import pandas as pd
import os

class PseStockPrice():

    def __init__(self):
        self.dir = r'C:\Users\ojell\Desktop\Oj\_Thesis\Data\stock_price'
        self.pse = 'pse_company_all.xls'
        self.api_url = r'https://pselookup.vrymel.com/api'
        self.period = ['2000-01-01', '2021-07-30']


    def pseFile(self):
        file = os.path.join(self.dir, self.pse)
        if os.path.exists(file):
            output = pd.read_excel(file)
        else:
            output = f"Check if {self.pse} exist"

        return output

    def pseAPI(self):
        df = self.pseFile()
        data = df.copy()    #Remove .head() once you've created the error function

        symbol = data['Stock Symbol']
        df_all = pd.DataFrame()
        for i in range(0, len(data)):
            response = requests.get(self.api_url + f'/stocks/{symbol[i]}/history/{self.period[0]}/{self.period[1]}')
            api = pd.DataFrame(response.json()['history'])
            comp = pd.concat([pd.DataFrame(data.iloc[i, :]).T] * len(api), ignore_index=True)
            comb_data = pd.concat([comp, api], axis=1)

            #comb_data.to_csv(self.dir + f'\{symbol[i]}_historical_data.csv', index=False)

            # NOTE: Commented out since there's an error when concatenating all data
            df_all = df_all.append(comb_data)

        # NOTE: Commented out since there's an error when concatenating all data
        df_all.to_csv(self.dir + r'\pse_stock_price_all.csv', index=False)

        msg = "Done pulling out data!"

        return msg


if __name__ == "__main__":
    test = PseStockPrice()
    #print(test.pseAPI())   # --> COMMENT OUT IF YOU WANT TO PULL OUT THE DATA

In [116]:
# NEWS 1: Get news articles and headline using requests, lxml and Beautifulsoup module from
#         Business World: https://www.bworldonline.com/
#         Inquirer.Net: https://www.inquirer.net/
#         BusinessMirror: https://businessmirror.com.ph/
#         Philippine News Agency: https://www.pna.gov.ph/
#         The Manila Times: https://www.manilatimes.net/
#         CNN PH: https://cnnphilippines.com/
#         PhilStar: https://www.philstar.com/
#         Manila Bulletin: https://mb.com.ph/

from bs4 import BeautifulSoup
import requests
import pandas as pd


class bwOnline():
    
    def __init__(self, news_url, category, page):
        self.n_url = news_url
        self.cat = category
        self.pg = page
    
    def parse(self, url):
        headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
        cont = requests.get(url, headers).content
        soup = BeautifulSoup(cont, 'html.parser')
        return soup
        
    def categoryUrl(self):
        sp = self.parse(self.n_url).find("ul", {"id": "menu-header-menu-1"}).find_all("a")
        
        ctgry_url = []            
        for item in range(0, len(sp)):
            if len(self.cat) == 1 and self.cat[0].lower() == 'all':
                for i in range(0, len(self.cat)):
                    ctgry_url.append(sp[item].get("href"))
            else:    
                for i in range(0, len(self.cat)):
                    if sp[item].text.lower() == self.cat[i].lower():
                        ctgry_url.append(sp[item].get("href"))
        
        return ctgry_url
        
    def articleUrl(self):         
        ctgry = self.categoryUrl()       
        page = self.pg      
        
        pgurl = []
        art_url = []
        for i in range(0, len(ctgry)):
            if page == 'all': # get back on this
                for a in range(1, 458):
                    pgurl.append(f'{ctgry[i]}page/{a}/')
            elif len(page) == 2:
                for b in range(page[0], page[1] + 1):
                    pgurl.append(f'{ctgry[i]}page/{b}/')
            else:
                for c in range(0, len(page)):
                    pgurl.append(f'{ctgry[i]}page/{page[c]}/')
            
        for url_i in pgurl:
            sp = self.parse(url_i).find("div", {"class": "td-ss-main-content"}).find_all("div", {"class": "item-details"})
            for item in sp:
                art_url.append(item.find("a").get("href"))
        
        return art_url
    
    def headlines(self):
        url = self.articleUrl()
        head = []
        for i in range(0, len(url)):
            head.append(self.parse(url[i]).find("h1", {"class": "entry-title"}).text)
        
        return head
    
    def published(self):
        url = self.articleUrl()
        pub_date = []
        for i in range(0, len(url)):
            # For now focus on date, not time
            pub_date.append(self.parse(url[i]).find("span", {"class": "td-post-date"}).text.split(" | ")[0])
        
        return pub_date
    
    def content(self):
        url = self.articleUrl()
        test = url[0]
        
        for i in range(0, len(url)):
            content = self.parse(test).find("div", {"class": "td-post-content td-pb-padding-side"}).text
        
        return content
    

if __name__ == "__main__":
    
    news_url = r'https://www.bworldonline.com/'
    category = ["Corporate"]
    page = [1]
    #date = []
    
    news = bwOnline(news_url, category, page)
    
    #test = news.content()
    

In [134]:
# NEWS 2: Modification of above script

from bs4 import BeautifulSoup
import requests
import pandas as pd


class bwOnline():
    
    def __init__(self, news_url, category, page):
        self.n_url = news_url
        self.cat = category
        self.pg = page
    
    def parse(self, url):
        headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
        cont = requests.get(url, headers).content
        soup = BeautifulSoup(cont, 'html.parser')        
        return soup
        
    def categoryUrl(self):
        sp = self.parse(self.n_url).find("ul", {"id": "menu-header-menu-1"}).find_all("a")
        ctgry_url = []
        for item in range(0, len(sp)):
            if len(self.cat) == 1 and self.cat[0].lower() == 'all':
                for i in range(0, len(self.cat)):
                    ctgry_url.append(sp[item].get("href"))
            else:    
                for i in range(0, len(self.cat)):
                    if sp[item].text.lower() == self.cat[i].lower():
                        ctgry_url.append(sp[item].get("href"))
        
        return ctgry_url
    
        
    def articleUrl(self):         
        ctgry = self.categoryUrl()       
        page = self.pg      
        pgurl = []
        art_url = []
        for i in range(0, len(ctgry)):
            #lastpg = self.parse(ctgry[i]).find("a", {"class": "last"}).text
            
            if page == 'all':
                for a in range(1, 458):
                    pgurl.append(f'{ctgry[i]}page/{a}/')
            elif len(page) == 2:
                for b in range(page[0], page[1] + 1):
                    pgurl.append(f'{ctgry[i]}page/{b}/')
            else:
                for c in range(0, len(page)):
                    pgurl.append(f'{ctgry[i]}page/{page[c]}/')
            
        for url_i in pgurl:
            sp = self.parse(url_i).find("div", {"class": "td-ss-main-content"}).find_all("div", {"class": "item-details"})
            for item in sp:
                art_url.append(item.find("a").get("href"))
        
        return art_url
    
    def category(self):
        sp = self.parse(self.n_url).find("ul", {"id": "menu-header-menu-1"}).find_all("a")
        
        ctgry = []
        for item in range(0, len(sp)):
            if len(self.cat) == 1 and self.cat[0].lower() == 'all':
                for i in range(0, len(self.cat)):
                    ctgry.append(sp[item].text)
            else:    
                for i in range(0, len(self.cat)):
                    if sp[item].text.lower() == self.cat[i].lower():
                        ctgry.append(sp[item].text)        
        return ctgry
    
    def headlines(self):
        url = self.articleUrl()
        head = []
        for i in range(0, len(url)):
            head.append(self.parse(url[i]).find("h1", {"class": "entry-title"}).text)
        
        return head
    
    def published(self):
        url = self.articleUrl()
        pub = []
        for i in range(0, len(url)):
            # For now focus on date, not time
            pub.append(self.parse(url[i]).find("span", {"class": "td-post-date"}).text.split(" | "))
        
        return pub
    
    def content(self):
        url = self.articleUrl()
        test = url[0]
        
        for i in range(0, len(url)):
            content = self.parse(test).find("div", {"class": "td-post-content td-pb-padding-side"}).text
        
        return content
    

if __name__ == "__main__":
    
    news_url = r'https://www.bworldonline.com/'
    category = ["Corporate"]
    page = [1]
    #date = []
    
    news = bwOnline(news_url, category, page)
    
    test = news.headlines()
    

   
    

In [57]:
# NEWS 3: Modification of above script

from bs4 import BeautifulSoup
import requests
import pandas as pd
import os


class parseNews():
    
    pass


class bwOnline():
    
    def __init__(self, news_url, category):
        self.n_url = news_url
        self.ctgry = category
    
    def parse(self, url):
        headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
        cont = requests.get(url, headers).content
        soup = BeautifulSoup(cont, 'html.parser')        
        return soup
        
    def allCategory(self):
        ct_sp = self.parse(self.n_url).find("ul", {"id": "menu-header-menu-1"}).find_all("a")
        
        # NOTE: Work on how to pull-out Health category
        exclude = ["BUYING RATES", "FOREIGN INTEREST RATES", "Philippine Mutual Funds", "Leaders and Laggards", 
                   "Stock Quotes", "Stock Markets Summary", "Non-BSP Convertible Currencies", "BSP Convertible Currencies",
                   "Health"]
        
        all_ctgry = [i.text for i in ct_sp if i.text not in exclude]
        all_ctgry_url = [i.get("href") for i in ct_sp if i.text not in exclude]
        lstpg = [self.parse(all_ctgry_url[i]).find("a", {"class": "last"}).text for i in range(0, len(all_ctgry_url))]
        
        dct = {"Category": all_ctgry, "Category Url": all_ctgry_url, "Last Page": lstpg}
        output = pd.DataFrame(dct)
        
        return output
    
        
    def headlines(self):
        allctgry = self.allCategory()
        
        #idx = data_df[data_df['Date']==p].index[0]
        output = allctgry[allctgry["Category"]==self.ctgry].index[0]
    
        
        
        return output
            
            
    

if __name__ == "__main__":
    
    news_url = r'https://www.bworldonline.com/'
    save_path = r'C:\Users\ojell\Desktop\Oj\_Thesis\Data\news'
    get_ctgry = ["Corporate"]
    #dt_range = []
    
    news = bwOnline(news_url, get_ctgry)
    
    test = news.headlines()    
    print(test)
    #output = test.to_excel(os.path.join(save_path, "news_output.xlsx"), index=False)
    
    

ValueError: ('Lengths must match to compare', (22,), (1,))